<h1><span style="color:blue"> Selecciona un conjunto de datos de los múltiples repositorios
existentes correspondiente a un problema de clasificación múltiple </span></h1>

In [1]:
import numpy as np
import matplotlib
import pandas as pd
from pandas import read_csv 
from sklearn.svm import SVC 

from sklearn.neural_network import MLPClassifier 

from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score 

from sklearn.model_selection import RepeatedStratifiedKFold 
from sklearn.model_selection import GridSearchCV 
from random import seed  
import time, random 

seed=random.seed(time.time()) 


Voy a utilizar el dataset **“Optical Recognition of Handwritten Digits”.**

Ya viene dividido en dos ficheros: uno para train y otro para test.

In [2]:
filename = 'train.csv'  
col_names=['pixel_0_0', 'pixel_0_1', 'pixel_0_2', 'pixel_0_3', 'pixel_0_4', 'pixel_0_5', 'pixel_0_6', 'pixel_0_7', 'pixel_1_0', 'pixel_1_1', 'pixel_1_2', 'pixel_1_3', 'pixel_1_4', 'pixel_1_5', 'pixel_1_6', 'pixel_1_7', 'pixel_2_0', 'pixel_2_1', 'pixel_2_2', 'pixel_2_3', 'pixel_2_4', 'pixel_2_5', 'pixel_2_6', 'pixel_2_7', 'pixel_3_0', 'pixel_3_1', 'pixel_3_2', 'pixel_3_3', 'pixel_3_4', 'pixel_3_5', 'pixel_3_6', 'pixel_3_7', 'pixel_4_0', 'pixel_4_1', 'pixel_4_2', 'pixel_4_3', 'pixel_4_4', 'pixel_4_5', 'pixel_4_6', 'pixel_4_7', 'pixel_5_0', 'pixel_5_1', 'pixel_5_2', 'pixel_5_3', 'pixel_5_4', 'pixel_5_5', 'pixel_5_6', 'pixel_5_7', 'pixel_6_0', 'pixel_6_1', 'pixel_6_2', 'pixel_6_3', 'pixel_6_4', 'pixel_6_5', 'pixel_6_6', 'pixel_6_7', 'pixel_7_0', 'pixel_7_1', 'pixel_7_2', 'pixel_7_3', 'pixel_7_4', 'pixel_7_5', 'pixel_7_6', 'output'] 
data = read_csv(filename, names=col_names)  

data.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,output
0,1,6,15,12,1,0,0,0,7,16,...,0,0,0,6,14,7,1,0,0,0
0,0,10,16,6,0,0,0,0,7,16,...,0,0,0,10,16,15,3,0,0,0
0,0,8,15,16,13,0,0,0,1,11,...,0,0,0,9,14,0,0,0,0,7
0,0,0,3,11,16,0,0,0,0,5,...,0,0,0,0,1,15,2,0,0,4
0,0,5,14,4,0,0,0,0,0,13,...,0,0,0,4,12,14,7,0,0,6


<h1><span style="color:blue">  Implementa un flujo de trabajo que incluya:
 </span></h1>
 
1. Imputación de valores perdidos.

2. Normalización de datos.

3. Ajuste de hiperparámetros mediante búsqueda grid.

4. Entrenamiento de una red neuronal mediante validación
cruzada.

<span style="color:blue">**1. Imputacion de valores perdídos**</span>.

#### Filosofía PANDAS

Se reemplazan los valores con NaN

Marcará todas aquellas situaciones dentro del dataframe con un **NaN** para saber que hay hay un valor perdido

Necesitamos trabajar esos datos faltantes

No se puede quedar asi, porque sino se leerian como datas categóricos

Los identificamos primero como **Nan** y ya sabremos que hacer con ellos

In [3]:
# Los convertimos a NaN
data[data=='?'] = np.nan
data.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,output
0,1,6,15,12,1,0,0,0,7,16,...,0,0,0,6,14,7,1,0,0,0
0,0,10,16,6,0,0,0,0,7,16,...,0,0,0,10,16,15,3,0,0,0
0,0,8,15,16,13,0,0,0,1,11,...,0,0,0,9,14,0,0,0,0,7
0,0,0,3,11,16,0,0,0,0,5,...,0,0,0,0,1,15,2,0,0,4
0,0,5,14,4,0,0,0,0,0,13,...,0,0,0,4,12,14,7,0,0,6


Lo primero que hago es identificar si existe un desequilibrio.
Utilizo la clase **groupby** para ver cuantos datos hay de cada clase.

In [4]:
class_distribution = data.groupby('output').size()
print(class_distribution)

output
0    376
1    389
2    380
3    389
4    387
5    376
6    377
7    387
8    380
9    382
dtype: int64


Ahora realizo un conteo del número de datos perdidos en cada columna

Después genero un bucle for que me cree un diccionario de diccionarios. En él guardo una lista de las opciones de cda columna.

Aplico la fromula **isna** y sumo todos los valores por columna. Así obtengo el resultado de si hay o no valores faltantes.

In [5]:
data1 = read_csv(filename, names=col_names)

counter = {}
for col_name in data1.columns: 
    counter[col_name] = []

for i in counter:
    # Con la funcion isna()----> se leen los datos perdidos en Pandas
    # Los adjuntamos a un disccionario para poder leerlos en un tabla
    counter[i].append(data[i].isna().sum())


Presento mi dataset con la información sobre valores faltantes.

In [6]:
df_counter = pd.DataFrame(counter)

df_counter


,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,output
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


No puedo simplememte eliminar la fila o la columna porque reduzco mi capacidad discriminante

Necesito encontrar una estrategia mejor para aplicar la eliminación de valores faltantes

Una vez identificados los patrones como NaN, pordemos utilizar el método **drona**

In [7]:
# -------------------------------#
# Aquellas FILAS que tienen al menos dos valores faltantes
# dropna(thresh=2)

# Aquellas COLUMNAS que tienen almenos dos valores faltantes.
# dropna(axis=1,thresh=2)

# Aquellas columnas que tengas al menos UN valor flatante
#clean_data.dropna(axis =1, inplace=True)

# Aquellas filas que tienen todos los valores faltantes
#clean_data.dropna(how= 'all')

# -------------------------------#


In [8]:
df_remove = pd.DataFrame(data)

# Aquellas filas que tienen todos los valores faltantes
df_remove.dropna(how= 'all')

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,output
0,1,6,15,12,1,0,0,0,7,16,...,0,0,0,6,14,7,1,0,0,0
0,0,10,16,6,0,0,0,0,7,16,...,0,0,0,10,16,15,3,0,0,0
0,0,8,15,16,13,0,0,0,1,11,...,0,0,0,9,14,0,0,0,0,7
0,0,0,3,11,16,0,0,0,0,5,...,0,0,0,0,1,15,2,0,0,4
0,0,5,14,4,0,0,0,0,0,13,...,0,0,0,4,12,14,7,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,5,13,11,2,0,0,0,2,15,...,0,0,0,8,13,15,10,1,0,9
0,0,0,1,12,1,0,0,0,0,0,...,0,0,0,0,4,9,0,0,0,4
0,0,3,15,0,0,0,0,0,0,11,...,0,0,0,4,14,16,9,0,0,6
0,0,6,16,2,0,0,0,0,0,15,...,0,0,0,5,16,16,16,5,0,6


En caso que este programa se ejecutara de manera automatiza, habría una condición para ejecutar un imputación Univariante y Multivariante de los datos.

<span style="color:red">UNI-VARIANTE</span>

Utilizar un método para reemplazar los valroes perdidos

- Estadístico representativo ----> Moda, Media, Mediana
- **Imputar un valor estimado por otro modelo predictivo**

Uno de los modelos más sensillos para econtrar valor perdido en **KNN imputer**

Busca los patrones que más cercanos através de una distancia euclídea

**1-NN-imputer**  Cuando encuentro el valor más cercano lo **INPUTO** en la posición del valor faltante.

**3-NN-imputer** buscaria los tres valores más cercanos y hacer una media de eso tres valores



In [9]:
from sklearn.impute import SimpleImputer

**SimpleImputer** utiliza los datos de entrenamiento para generar un modelo de imputación

In [10]:
# Lo primero es crear el objeto
# La estrategia se puede cambiar denpendiendo si necesitamos que cuente - str -
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Alimento el modelo de imputación con los datos de entrenamiento
imputer.fit(df_remove)

# Aplica el modelo de imputación recien creado sobre los datos que se le pasen
newdata = imputer.transform(df_remove)

df_remove.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,output
0,1,6,15,12,1,0,0,0,7,16,...,0,0,0,6,14,7,1,0,0,0
0,0,10,16,6,0,0,0,0,7,16,...,0,0,0,10,16,15,3,0,0,0
0,0,8,15,16,13,0,0,0,1,11,...,0,0,0,9,14,0,0,0,0,7
0,0,0,3,11,16,0,0,0,0,5,...,0,0,0,0,1,15,2,0,0,4
0,0,5,14,4,0,0,0,0,0,13,...,0,0,0,4,12,14,7,0,0,6


<span style="color:red">MULTI-VARIANTE</span>

In [11]:
from sklearn.impute import KNNImputer

In [12]:
multi_data  = df_remove.copy()

In [13]:
# Primero creo un objeto Imputer
imputer = KNNImputer()


# Despues le paso el dataframe completo
imputer.fit(multi_data)

KNNImputer()

In [14]:
# Lo que hace e sgenerar un nuevo dataframe sin los valores perdido utilizamos este metodo de KNN
# Utiliza la metrica **Euclidea**
new_data = imputer.transform(multi_data)

new_data

array([[ 1.,  6., 15., ...,  0.,  0.,  0.],
       [ 0., 10., 16., ...,  0.,  0.,  0.],
       [ 0.,  8., 15., ...,  0.,  0.,  7.],
       ...,
       [ 0.,  3., 15., ...,  0.,  0.,  6.],
       [ 0.,  6., 16., ...,  5.,  0.,  6.],
       [ 0.,  2., 15., ...,  0.,  0.,  7.]])

<span style="color:blue">**2. Normalización de datos**</span>.

<span style="color:green"> **Primero** </span> 

Divido los datos entre los inputs y el output

En este caso tengo dos data frames, uno para entrenamiento otro de pureba.

Sin embargo, igual lo voy a dividir para crear un subconjunto de prueba dentro de training para ajustar los hiperparametros.



<span style="color:green"> **Segundo** </span> 

Divido con la función **train_test_split** para tner un subconjunto para el ajuste de hiperparametros. 

Pero la la validación utilizo directamente el  <span style="color:red"> **X** y **Y** </span> del conjunto de test.

In [15]:
X = data[data.columns[:-1]] 
Y = data['output']  

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state=seed)

<span style="color:green">Escalado de datos</span>

Separar las variables que sean de tipo numerico de las categoricas.

Y con **EstandardScaler** le aplico un escaldo a esas variables numericas.

Aqui sigo trabajando unicamente con el set de training que dividí en dos , para trabajar despues con la optimizacion de parametros


In [17]:
#### **NUMERICAS** ---------------> StandardScaler
num_X_train = X_train.select_dtypes(include=['int','float'])
num_X_test = X_test.select_dtypes(include=['int','float'])

In [18]:
stdScaler = StandardScaler()
stdScaler.fit(num_X_train)

StandardScaler()

In [19]:
resc_X_train = pd.DataFrame(stdScaler.transform(num_X_train),columns=num_X_train.columns)

resc_X_test = pd.DataFrame(stdScaler.transform(num_X_test),columns=num_X_test.columns)

En este caso no tengo variables categoricas, entonces no es necesario realizar un reescalado con **la clase ONEHOT ENCODER**

In [20]:
#### **CATEGORICAS** ---------------> OneHotEncoding

#La funcion **.select_dtypes()** devuelve un dataframe
#cat_X_train = X_train.select_dtypes(include=['object'])
#cat_X_test = X_test.select_dtypes(include=['object'])
# ohe = OneHotEncoder(drop = 'if_binary')
# ohe.fit(cat_X_train)
#resc_X_train[ohe.get_feature_names_out()] = ohe.transform(cat_X_train).toarray()
#resc_X_test[ohe.get_feature_names_out()] = ohe.transform(cat_X_test).toarray()

De esta manera genero un modelo de transformación

Es decir un nuevo dataframe con los datos ya trasformados:

- <span style="color:red">Los datos Categoricos pasados a numericos</span>
- <span style="color:red">Los datos Numericos escalados</span>

In [21]:
resc_X_train.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_5,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6
0,-0.343923,-0.748877,-0.437316,0.343636,-0.094309,-0.411625,-0.132985,-0.022753,-0.636502,0.449978,...,-0.759524,-0.200849,-0.019335,-0.298566,-0.971300,0.943326,0.714458,0.050585,-0.526288,-0.180124
1,-0.343923,-1.178827,-0.673828,0.122586,-0.094309,-0.411625,-0.132985,-0.022753,-0.636502,0.819403,...,-0.759524,-0.200849,-0.019335,-0.298566,-1.171886,-0.449798,0.117428,-0.295175,-0.526288,-0.180124
2,-0.343923,-0.533903,-0.910339,-0.319515,1.150649,4.046719,-0.132985,-0.022753,-0.636502,-0.473585,...,-0.759524,-0.200849,-0.019335,-0.298566,-0.770713,0.246764,-2.270692,-1.159574,-0.526288,-0.180124
3,-0.343923,-1.178827,-2.329407,-0.319515,1.862053,3.155050,1.833132,-0.022753,-0.636502,-1.581859,...,-0.759524,-0.200849,-0.019335,-0.298566,-1.171886,-2.771670,0.515448,0.396345,-0.526288,-0.180124
4,-0.343923,-1.178827,-2.329407,1.006787,-0.272160,-0.411625,-0.132985,-0.022753,-0.636502,-1.951284,...,-0.759524,-0.200849,-0.019335,-0.298566,-1.171886,-2.539483,0.714458,-0.122295,-0.526288,-0.180124


<span style="color:blue">**3. Ajuste de hiperparámetros mediante búsqueda grid.**</span>.

Aqui tenemos que definir el modelo de clasificación

Lo primero es definiré el modelo de clasificación cruzada

Entonces voy a utilizar una clase que tambien está incluida dentro de model_selection. **RepeatedStratifiedKFold**

Lo que hace es repetir un KFold 10 3 veces, es decir un total de 30

Como es un problema de clasificacón le digo que es una selección estrateficadas.

Que me asegure que cada uno de los FOLD tienen un equilibrio de clase igual que el datset original.

- Esto es muy importante

In [22]:
from pandas import read_csv 
from sklearn.svm import SVC 
from sklearn.model_selection import RepeatedStratifiedKFold 
from sklearn.model_selection import GridSearchCV 
from random import seed  
from time import time  
seed= seed(time()) 

In [23]:
model = MLPClassifier() 
cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3 ) 

Ahora definimos el espacio en donde queremos que cada un o de los hiper parametros busque

Definimos el espacio de busqueda

Realizamos una lista de diccionario

Cada dict va a contener la palabra clave y dentro una lista con los distintos valores que puede tomar ese hiper parametro.

No los costumizo todos porque tiene un alto coste conputacional.

Sin embargo, los dejo disponibles como comentarios.

In [24]:
# Esto son todos los hiper parametros de una red neuronal
space =  [{'hidden_layer_sizes' :[50,100],
           'max_iter': [10000],
           'activation':['identity', 'logistic', 'relu','tanh'],
           #'n_iter_no_change':[25, 50],
           'solver': ['lbfgs','sgd','adam'],
           #'random_state': [0,1,2,3],
           #'learning_rate':['constant', 'invscaling', 'adaptive'],         
           #'alpha':[1e-5, 0.1, 1]
          } 
         ] 

Ahora definimos un objeto y aqui indico el modelo que definí más arriba

Indico el espacio de busqueda (grid)

Indico la métrica a optimizar

In [25]:
search = GridSearchCV(model,
                      param_grid=space,
                      scoring='balanced_accuracy',
                      n_jobs=-1,
                      cv=cv) 

Por último invoco al método fit y me realiza todo el procedimiento que previamente he definido.

Con son dos datasets, el set de train lo dividí en dos . EL subconjunto test es con el que voy a probar los hiper parametros.

In [26]:
result = search.fit(resc_X_test,y_test)

print(result.best_score_)

0.9672281607281605


Esta información es la que necesito a la hora de entrenar mi modelo más adelante.

In [27]:
print(result.best_params_)

{'activation': 'relu', 'hidden_layer_sizes': 100, 'max_iter': 10000, 'solver': 'adam'}


<span style="color:blue">**4. Entrenamiento de una red neuronal mediante validación cruzada.**</span>.

Aqui trabajo con los dos datasets (training y test)


In [28]:
filename = 'test.csv'  
col_names=['pixel_0_0', 'pixel_0_1', 'pixel_0_2', 'pixel_0_3', 'pixel_0_4', 'pixel_0_5', 'pixel_0_6', 'pixel_0_7', 'pixel_1_0', 'pixel_1_1', 'pixel_1_2', 'pixel_1_3', 'pixel_1_4', 'pixel_1_5', 'pixel_1_6', 'pixel_1_7', 'pixel_2_0', 'pixel_2_1', 'pixel_2_2', 'pixel_2_3', 'pixel_2_4', 'pixel_2_5', 'pixel_2_6', 'pixel_2_7', 'pixel_3_0', 'pixel_3_1', 'pixel_3_2', 'pixel_3_3', 'pixel_3_4', 'pixel_3_5', 'pixel_3_6', 'pixel_3_7', 'pixel_4_0', 'pixel_4_1', 'pixel_4_2', 'pixel_4_3', 'pixel_4_4', 'pixel_4_5', 'pixel_4_6', 'pixel_4_7', 'pixel_5_0', 'pixel_5_1', 'pixel_5_2', 'pixel_5_3', 'pixel_5_4', 'pixel_5_5', 'pixel_5_6', 'pixel_5_7', 'pixel_6_0', 'pixel_6_1', 'pixel_6_2', 'pixel_6_3', 'pixel_6_4', 'pixel_6_5', 'pixel_6_6', 'pixel_6_7', 'pixel_7_0', 'pixel_7_1', 'pixel_7_2', 'pixel_7_3', 'pixel_7_4', 'pixel_7_5', 'pixel_7_6', 'output'] 
df_test = read_csv(filename, names=col_names)  

df_test.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,output
0,0,5,13,9,1,0,0,0,0,13,...,0,0,0,6,13,10,0,0,0,0
0,0,0,12,13,5,0,0,0,0,0,...,0,0,0,0,11,16,10,0,0,1
0,0,0,4,15,12,0,0,0,0,3,...,0,0,0,0,3,11,16,9,0,2
0,0,7,15,13,1,0,0,0,8,13,...,0,0,0,7,13,13,9,0,0,3
0,0,0,1,11,0,0,0,0,0,0,...,0,0,0,0,2,16,4,0,0,4


#### Primero

Divido los datos de **test** y **train** entre los inputs y el output

In [29]:
# Recordemos que aqui debemos utilizar el set completo de TRAIN
X_train1 = X
y_train1 = Y 

X_test1 = df_test[df_test.columns[:-1]] 
y_test1 = df_test['output']

<span style="color:red">Escalado de datos (Normalización)</span>

Separar las variables que sean de tipo numerico de las categoricas.
Y con **EstandardScaler** le aplico un escaldo a esas variables numericas.

#### **NUMERICAS** ---------------> StandardScaler

In [30]:
scaler =StandardScaler()

X_train = scaler.fit_transform(X_train1) 
X_test= scaler.transform(X_test1)

**OneHotEncoder** para las variables **CATEGORICAS**

In [31]:
from sklearn.preprocessing import OneHotEncoder

#La funcion **.select_dtypes()** devuelve un dataframe
cat_X_train = X_train1.select_dtypes(include=['object'])
cat_X_test = X_test1.select_dtypes(include=['object'])


In [32]:
ohe = OneHotEncoder(drop = 'if_binary')
ohe.fit(cat_X_train)


OneHotEncoder(drop='if_binary')

Ahora ya tenemos el dataset equilibrado para poder trabajarlo

In [33]:
resc_X_train[ohe.get_feature_names_out()] = ohe.transform(cat_X_train).toarray()
resc_X_test[ohe.get_feature_names_out()] = ohe.transform(cat_X_test).toarray()

resc_X_test.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_5,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6
0,-0.343923,-0.103953,0.745241,0.785737,0.083542,-0.411625,-0.132985,-0.022753,-0.636502,0.449978,...,-0.759524,-0.200849,-0.019335,-0.298566,0.031632,0.711138,-0.280592,-0.986695,-0.526288,-0.180124
1,-0.343923,0.540972,0.981752,-0.761616,-0.983564,-0.411625,-0.132985,-0.022753,-0.309702,1.004115,...,0.060937,-0.200849,-0.019335,-0.298566,0.031632,0.711138,0.515448,-0.295175,-0.526288,-0.180124
2,-0.343923,-0.963852,0.035706,1.006787,0.261393,-0.411625,-0.132985,-0.022753,-0.636502,-0.473585,...,1.496744,-0.200849,-0.019335,-0.298566,-1.171886,0.014577,0.913468,1.606504,1.458347,-0.180124
3,-0.343923,-1.178827,-0.910339,0.785737,-0.805713,-0.411625,-0.132985,-0.022753,-0.636502,-1.766572,...,0.676283,-0.200849,-0.019335,-0.298566,-1.171886,-1.146359,0.714458,1.606504,2.698744,-0.180124
4,3.064735,2.260771,0.745241,-0.982667,-0.983564,-0.411625,-0.132985,-0.022753,0.997496,1.004115,...,-0.759524,-0.200849,-0.019335,1.857877,1.836909,0.014577,-1.673662,-1.159574,-0.526288,-0.180124


<span style="color:red">Entreno el modelo</span>

#### Leave-one-out

Siempre dejo un patron afuentra para TESt y voy rotando ese patron hasta que todos los patrones hayan sido una vez TEST

Generar tantos modelos no es fácil y toma muchisimos recurso computacionales.

Pero utilizo este metodo porque es el que  más se parece la vida real.


Es para que la estimacion se haga con patrones random independendientes.

Hay tres tipos:
    
    - K-fold
    - Leaveoneout
    - Monte Carlo

In [34]:
## Son los mismos imports de arriba más éste
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score

from pandas import read_csv 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

Aquí es don de utilizo la linformacion que obtuve del subconjunto de prueba. Utilizo los hiper parametros otimos para este modelo de redes neuronales.

In [35]:
model = MLPClassifier(hidden_layer_sizes=(result.best_params_['hidden_layer_sizes'],), max_iter=result.best_params_['max_iter'], activation=result.best_params_['activation'], n_iter_no_change=50, solver=result.best_params_['solver']).fit(resc_X_train, y_train) 
loocv = LeaveOneOut()


In [ ]:


# Igual que antes lo unico es introducir el objeto loovp dentro del parametro CV (Cross validation)
results = cross_val_score(model, resc_X_train,y_train, cv=loocv)

## Aqui muestro la MEDIA y la DESVIACION STD para registrar el rendimiento de estemodelo
print("Acc: %.3f+/.%.3f"% (results.mean(),results.std()))

<h1><span style="color:blue"> Reporta la mejor combinación posible de hiperparámetros y la
correspondiente métrica de evaluación de su rendimiento. </span></h1>